### **Mike Stebbins, 2019-07-17**

#### Calculate a factor to apply to an RSS summation of tolerances that will be as conservative as a 2 or 3-sigma value obtained from a Monte Carlo simulation

Cases reviewed:
* A: same magnitude tolerances: *i.e., ± 5, ± 5, ± 5, ± 5, ± 5 microns*
* B: varying magnitude tolerances: *i.e., ± 1, ± 2, ± 3, ± 4, ± 5 microns*
* C: tolerances are almost uniform, except for one large outlier: *i.e., ± 1, ± 1, ± 1, ± 1, ± 5 microns*

In [228]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as random
from scipy.stats import shapiro, normaltest

In [229]:
def return_random_tolerance(low_value,high_value):
    if (distribution_type == 0):
        high_bound = high_value * 1.0001
        return random.uniform(low_value,high_bound,1)
    if (distribution_type == 1):
        location = (high_value+low_value)/2.0
        #print("Location = ",location)
        scale = (high_value-(high_value+low_value)/2)/3.0
        #print ("Scale = ",scale)
        return random.normal(location,scale,1)

In [230]:
def random_tolerance_summing(list_of_ranges):
    total_sum = 0.0
    for each in list_of_ranges:
        temp = return_random_tolerance(each[0],each[1])
        total_sum = total_sum + temp
    return total_sum

In [231]:
def worst_case_summing(list_of_ranges):
    total_sum = 0.0
    for each in list_of_ranges:
        temp = each[1]
        total_sum = total_sum + temp
    return total_sum

In [232]:
def rss_summing(list_of_ranges):
    total_sum = 0.0
    for each in list_of_ranges:
        temp = (each[1])**2
        total_sum = total_sum + temp
    return (total_sum)**(0.5)

In [233]:
def monte_carlo_summing(list_of_ranges):
    monte_tolerance_sums = []
    for i in range (0,number_of_iterations):
        temp = random_tolerance_summing(list_of_ranges)
        monte_tolerance_sums.append(temp.item())
    return(monte_tolerance_sums)
    #print("monte_tolerance_sums length = ",len(monte_tolerance_sums))
    #plt.hist(monte_tolerance_sums, bins='auto')  # arguments are passed to np.histogram
    #plt.title("Monte Carlo Tolerance Summations with "+str(number_of_iterations)+" Iterations")
    #plt.show()
    #mean = np.mean(monte_tolerance_sums)
    #median = np.median(monte_tolerance_sums)
    #std_dev = np.std(monte_tolerance_sums)
    #print("Mean = ",mean)
    #print("Median = ",median)
    #print("Standard Deviation = ",std_dev)
    #print("3 * Standard Deviation = ",3*std_dev)

In [234]:
def compute_RSS_factor(worst,rss,std_dev):
    if ((sigma_value*std_dev) < worst):
        temp = (sigma_value*std_dev)/rss
    else:
        temp = worst/rss
    return temp

In [235]:
def run_the_case(tolerances):
    monte_carlo_sums = monte_carlo_summing(tolerances)
    monte_carlo_stdev = np.std(monte_carlo_sums)
    
    if (plot_stuff == 1):
        plt.hist(monte_carlo_sums, bins='auto')  # arguments are passed to np.histogram
        plt.title("Monte Carlo Tolerance Summations with "+str(number_of_iterations)+" Iterations")
        plt.show()
    
    worst_case_sum = worst_case_summing(tolerances)
    rss_sum = rss_summing(tolerances)    
    rss_factor = compute_RSS_factor(worst_case_sum,rss_sum,monte_carlo_stdev)  

    if (print_stuff == 1):
        print("Monte Carlo Summation Median = %.6f" % (np.median(monte_carlo_sums)))
        print("Monte Carlo Summation %.0f Sigma = %.6f" % (sigma_value,sigma_value*monte_carlo_stdev))
        print("Worst-Case Straight Summation = %.6f" % (worst_case_sum))
        print("RSS Summation = %.6f" % (rss_sum))
        print("Factor to get RSS up to either worst case or %.0f Sigma value =  %.3f" % (sigma_value,rss_factor))
        print("")
    
    return rss_factor

In [236]:
def run_many_cases(runs,tolerances):
    factors = []
    for i in range(0,runs):
        if (print_stuff == 1):
            print("Case #%.0f" % (i))
        temp = run_the_case(tolerances)
        factors.append(temp)
    factor_mean = np.mean(factors)
    factor_stdev = np.std(factors)
    print ("Mean / StDev of factors over %.0f Cases with %.0f Tolerances = %.6f / %.6f" % (runs,len(tolerance_ranges),factor_mean,factor_stdev))
    return (factor_mean,factor_stdev)

In [237]:
# USER-DEFINED VALUES THAT YOU MAY WANT TO CHANGE
sigma_value = 3 # 2 OR 3
distribution_type = 0  # 0 = Uniform tolerance distribution, 1 = Normal distribution

# VALUES THAT YOU COULD CHANGE, BUT PROBABLY DON'T NEED TO
number_of_runs = 100
number_of_iterations = 10000

# DEBUG TYPE PRINT-OUTS
print_stuff = 0  # 0 = don't print a bunch of stuff, 1 = print a bunch of stuff
plot_stuff = 0   # 0 = don't plot a bunch of stuff, 1 = plot a bunch of stuff

print("CASE A Results:")
tolerance_ranges = [(-5.0,5.0),(-5.0,5.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-5.0,5.0),(-5.0,5.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-5.0,5.0),(-5.0,5.0),(-5.0,5.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-5.0,5.0),(-5.0,5.0),(-5.0,5.0),(-5.0,5.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)
print("-------------------------------------------------")

print("CASE B Results:")
tolerance_ranges = [(-5.0,5.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-3.0,3.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-3.7,3.7),(-2.3,2.3),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-4.0,4.0),(-3.0,3.0),(-2.0,2.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)
print("-------------------------------------------------")

print("CASE C Results:")
tolerance_ranges = [(-5.0,5.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-1.0,1.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-1.0,1.0),(-1.0,1.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)

tolerance_ranges = [(-5.0,5.0),(-1.0,1.0),(-1.0,1.0),(-1.0,1.0),(-1.0,1.0)]
factor_mean, factor_stdev = run_many_cases(number_of_runs,tolerance_ranges)
print("-------------------------------------------------")

CASE A Results:
Mean / StDev of factors over 100 Cases with 2 Tolerances = 1.414214 / 0.000000


KeyboardInterrupt: 